In [1]:
# **Imports and Constants**

In [2]:
# imports
from __future__ import print_function
from pyspark import SparkContext
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from mpl_toolkits.mplot3d import Axes3D
from pyspark.sql.functions import col
from functools import reduce
from pyspark import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.sql.functions import col, split, size, isnan, array_contains, array_min, when, count
from pyspark.sql.types import StructType, StructField, TimestampType, StringType, FloatType
import pyspark.sql.functions as f
import matplotlib.pyplot as plt
import pandas as pd
import glob
import re
import pathlib
import json
import datetime
import numpy as np
import time
import os
import random
import statistics
import platform

In [6]:
# paths
DATASET_PATH = '/Users/Soroush/Desktop/Thesis/Code/Datasets/Smart*/apartment/'

# BASE_PATH=pathlib.Path().absolute()
# KMEANS_REL_PATH="kmeans models"
# DATASET_REL_PATH="dataset"
# DATASET_PATH=os.path.join(BASE_PATH,DATASET_REL_PATH)
# KMEANS_PATH=os.path.join(BASE_PATH,KMEANS_REL_PATH)

# from google.colab import drive
# drive.mount('/gdrive')

In [7]:
# env variables
if platform.system() == 'Windows':
    %env PYSPARK_DRIVER_PYTHON = python
    %env PYSPARK_PYTHON = python
elif platform.system() == 'Linux':
    %env PYSPARK_DRIVER_PYTHON = python
    %env PYSPARK_PYTHON = python3
else:
    %env PYSPARK_DRIVER_PYTHON = python3.6
    %env PYSPARK_PYTHON = python3.6

# incompatibility with Pyarrow
# need to install Pyarrow 0.14.1 or lower or Set the environment variable ARROW_PRE_0_15_IPC_FORMAT=1
%env ARROW_PRE_0_15_IPC_FORMAT = 1

env: PYSPARK_DRIVER_PYTHON=python3.6
env: PYSPARK_PYTHON=python3.6
env: ARROW_PRE_0_15_IPC_FORMAT=1


In [8]:
# load and save .read_pickle() and .to_pickle()

# save
# dataset.to_pickle(DATASET_PATH+"dataset.pkl")
# aggregated_dataset.to_pickle(DATASET_PATH+"aggregated_dataset.pkl")
# json_dataset.to_pickle(DATASET_PATH+"json_dataset.pkl")
# dataset.to_csv(DATASET_PATH+"dataset.csv")
# aggregated_dataset.to_csv(DATASET_PATH+"aggregated_dataset.csv")
# json_dataset.to_csv(DATASET_PATH+"json_dataset.csv")
# aggregated_dataset_rowBased.to_csv(DATASET_PATH+"aggregated_dataset_rowBased.csv")


# load
# dataset=pd.read_pickle(DATASET_PATH+"dataset.pkl")
# aggregated_dataset=pd.read_pickle(DATASET_PATH+"aggregated_dataset.pkl")
# json_dataset=pd.read_pickle(DATASET_PATH+"json_dataset.pkl")

In [27]:
# create appropriate dataset

# dataset address: http://traces.cs.umass.edu/index.php/Smart/Smart

# Extract File
# import tarfile
# !tar -xf '/gdrive/My Drive/a.gzip' -C '/gdrive/My Drive/'

# load


def load_smart_star_dataset(dataset_path):

    # length of file path
    LENGTH = len(dataset_path)+5  # (5 for removing 2014/, 2015/ and 2016/)

    df_merged = pd.DataFrame(columns=['date'])

    # 2014
    #path_2014 = r'/gdrive/My Drive/Dataset/apartment/2014'
    path_2014 = dataset_path+'2014'
    all_2014_paths = glob.glob(path_2014 + "/Apt*.csv")
    df_merged_2014 = pd.DataFrame(columns=['date'])
    for file_name in all_2014_paths:
        column_name = file_name[LENGTH:-4]  # (4 for .csv)
        # column_name = file_name.replace("dataset/2014/", "").replace("_2014.csv","")
        df = pd.read_csv(file_name, names=["date", column_name])
        df_merged_2014 = pd.merge(
            df_merged_2014, df, on='date', how='outer', left_index=True, right_index=True)

    # 2015
    df_merged_2015 = pd.DataFrame(columns=['date'])
    #path_2015 = r'/gdrive/My Drive/Dataset/apartment/2015'
    path_2015 = dataset_path+'2015'
    all_2015_paths = glob.glob(path_2015 + "/Apt*.csv")
    for file_name in all_2015_paths:
        column_name = file_name[LENGTH:-4]
        # column_name = file_name.replace("dataset/2015/", "").replace("_2015.csv","")
        df = pd.read_csv(file_name, names=["date", column_name])
        df_merged_2015 = pd.merge(
            df_merged_2015, df, on='date', how='outer', left_index=True, right_index=True)

    # 2016
    df_merged_2016 = pd.DataFrame(columns=['date'])
    #path_2016 = r'/gdrive/My Drive/Dataset/apartment/2016'
    path_2016 = dataset_path+'2016'
    all_2016_paths = glob.glob(path_2016 + "/Apt*.csv")
    for file_name in all_2016_paths:
        column_name = file_name[LENGTH:-4]
        # column_name = file_name.replace("dataset/2016/", "").replace("_2016.csv","")
        df = pd.read_csv(file_name, names=["date", column_name])
        df_merged_2016 = pd.merge(
            df_merged_2016, df, on='date', how='outer', left_index=True, right_index=True)

    # merge all years
    df_merged = df_merged_2014.append(df_merged_2015, ignore_index=True).append(
        df_merged_2016, ignore_index=True)
    final = df_merged
    # save
    return final


In [28]:
# aggregate data
def agg_by_date(final):
    final['date'] = pd.to_datetime(final['date'],)
    final.index = final['date']

    # for more than to NaN : .apply(lambda x: x.sum() if x.isnull().sum() <= 2 else np.nan)
    final_agg_by_hour = final.resample('60T').mean()

    final = final_agg_by_hour.resample('D').aggregate(
        lambda x: x.tolist())  # for tuple: .aggregate(lambda x: tuple(x))

    # remove first and last row
    # aggregated_dataset.drop(pd.to_datetime('2016-12-28'),inplace=True)
    # aggregated_dataset.drop(pd.to_datetime('2014-10-15'),inplace=True)
    # final=final.iloc[1:-1]

    return final

In [29]:
# convert column_based df to row_based
def col_to_row(df):
    # house_id=df.columns
    df.reset_index(inplace=True)
    return pd.melt(df, id_vars=['date'], value_name='power')

In [30]:
# create dataset
dataset = load_smart_star_dataset(DATASET_PATH)

# aggregate by date
aggregated_dataset = agg_by_date(dataset)

# aggregate by date, row-based
aggregated_dataset_rowBased = col_to_row(aggregated_dataset)
aggregated_dataset_rowBased.sort_values(
    ['date', 'variable'], ascending=[True, True], inplace=True)

In [31]:
# save
dataset.to_csv(DATASET_PATH+"dataset.csv")
aggregated_dataset.to_csv(DATASET_PATH+"aggregated_dataset.csv")
aggregated_dataset_rowBased.to_csv(
    DATASET_PATH+"aggregated_dataset_rowBased.csv")

dataset.to_pickle(DATASET_PATH+"dataset.pkl")
aggregated_dataset.to_pickle(DATASET_PATH+"aggregated_dataset.pkl")
aggregated_dataset_rowBased.to_pickle(
    DATASET_PATH+"aggregated_dataset_rowBased.pkl")